In [1]:
import matplotlib.pyplot as plt
import pathlib
import os
import cv2
import numpy as np
from random import shuffle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical


data_dir = "D:/Durhum/2 sem/Soil_Dataset"
IMG_SIZE = 32
    

Using TensorFlow backend.


In [2]:
#Go through all the folders and save the images & labels in list variable.
#return data is a list variable, it contain (image,label)
def get_data(folder):
    data = []
    folder_path = os.path.join(data_dir,folder)
    for img_fol in os.listdir(folder_path):
        label = str(img_fol)
        img_fol_path = os.path.join(folder_path,img_fol)
        for imgs in os.listdir(img_fol_path):
            imgs_folder = os.path.join(img_fol_path, imgs)
            img = cv2.imread(imgs_folder,cv2.IMREAD_COLOR)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            data.append([img,label])
    shuffle(data)
    return data

#Check for train dataset and test dataset and seperate them for model training and testing
def train_test_dataset():
    for folder in os.listdir(data_dir):
        if folder == "Train":
            train_data = get_data(folder)
        else:
            test_data = get_data(folder)
    return train_data, test_data


train_data, test_data = train_test_dataset()

In [3]:
#create train_x, train_y and test_x, test_y list
train_X = []
train_y = []
test_X = []
test_y = []
for i in train_data:
    train_X.append(i[0])
    train_y.append(i[1])

for i in test_data:
    test_X.append(i[0])
    test_y.append(i[1])

In [5]:
#Data preprocessing for image data.
#converting into float32 variable
train_X=np.array(train_X)
test_X=np.array(test_X)
 
train_X=train_X/255.0
test_X=test_X/255.0


In [6]:
#one-hot encoding for label data
def onehot(data):
    label_encoder=LabelEncoder()
    label_ids=label_encoder.fit_transform(data)
    onehot_encoder=OneHotEncoder(sparse=False)
    reshaped=label_ids.reshape(len(label_ids), 1)
    return onehot_encoder.fit_transform(reshaped)


In [7]:
train_y = np.array(onehot(train_y))

In [17]:
np.shape(train_y)

(715, 4)

In [9]:
test_y = np.array(onehot(test_y))

In [10]:
np.shape(test_y)

(188, 4)

In [11]:
#Sequential model
model=Sequential()
model.add(Conv2D(715,(3,3),input_shape=(32,32,3),
    padding='same',activation='relu',
    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [12]:
sgd=SGD(lr=0.01,momentum=0.9,decay=(0.01/25),nesterov=False)
 
model.compile(loss='categorical_crossentropy',
  optimizer=sgd,
  metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 715)       20020     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 715)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        205952    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)              

In [14]:
model.fit(train_X, train_y, validation_data=(test_X,test_y), epochs=10, batch_size=32, verbose=1)


Train on 715 samples, validate on 188 samples
Epoch 1/10
715/715 [==============================] - 19s 26ms/step - loss: 1.2205 - accuracy: 0.4378 - val_loss: 0.9588 - val_accuracy: 0.5691
Epoch 2/10
715/715 [==============================] - 22s 30ms/step - loss: 0.5187 - accuracy: 0.7748 - val_loss: 0.5486 - val_accuracy: 0.7340
Epoch 3/10
715/715 [==============================] - 23s 32ms/step - loss: 0.4704 - accuracy: 0.8028 - val_loss: 0.3720 - val_accuracy: 0.8298
Epoch 4/10
715/715 [==============================] - 23s 32ms/step - loss: 0.5287 - accuracy: 0.7818 - val_loss: 0.5601 - val_accuracy: 0.7713
Epoch 5/10
715/715 [==============================] - 23s 32ms/step - loss: 0.4006 - accuracy: 0.8531 - val_loss: 0.5242 - val_accuracy: 0.7926
Epoch 6/10
715/715 [==============================] - 23s 32ms/step - loss: 0.4151 - accuracy: 0.8182 - val_loss: 0.4528 - val_accuracy: 0.8298
Epoch 7/10
715/715 [==============================] - 22s 31ms/step - loss: 0.3133 - accu

In [15]:
_,acc=model.evaluate(test_X,test_y)
print(acc*100)

188/188 [==============================] - 1s 5ms/step
86.17021441459656


In [16]:
model.save("model_soil_testing_10epoch.h5")